### **Week 3,4: Function and Recursion** ###

#### 1. Example: Sound [(Video)](https://www.youtube.com/watch?v=TC_JcE42R2s&list=PL6BsET-8jgYVoDRPWXvw3q5ZsdpwVeEyY) ####

Why higher-order functions? We want to work in the space of functions! Take a look at the example below:  
**Example:** Write a .wav file. WAV format is simple but not compressed, not much used these days. We can aquisite .wav from nature or just generate wave signal.

![.wav File](../Sources/lec_06.png)

In [1]:
from wave import open
from struct import Struct
from math import floor

frame_rate = 11025

def encode(x):
    """
    Encode float x between -1 and 1 as 2 bytes.
    See library [struct]
    """
    i = int(16384*x)
    return Struct('h').pack(i)

def play(sampler, name='song.wav', seconds=2):
    """
    Write the output of a sampler function as a wav file.
    See library [wave]
    """
    out = open(name,'wb')
    out.setnchannels(1)
    out.setsampwidth(2)
    out.setframerate(frame_rate)
    t = 0

    while t < seconds*frame_rate:
        sample = sampler(t)
        out.writeframes(encode(sample))
        t = t+1
    out.close()

def tri(frequency, amplitude=0.3):
    """A continuous triangle wave."""
    period = frame_rate // frequency
    def sampler(t):
        saw_wave = t/period - floor(t/period + 0.5)
        tri_wave = 2*abs(2*saw_wave) - 1
        return amplitude*tri_wave
    return sampler

c_freq = 261.63


In [ ]:
# a demo to show triangle waves.
c = tri(c_freq)
t = 0
while t < 100:
    print(c(t))
    t +=1

play(c)

In [13]:
c_freq, e_freq, g_freq = 261.63, 329.63, 392.00

def both(f,g):
    return lambda t: f(t) + g(t)

play(both(tri(c_freq), tri(e_freq)))

def note(f, start, end, fade=0.01):
    def sampler(t):
        seconds = t/frame_rate
        if seconds < start:
            return 0
        elif seconds > end:
            return 0
        elif seconds < start + fade:
            return (seconds-start) / fade * f(t)
        elif seconds > end + fade:
            return (end - seconds) / fade * f(t)
        else:
            return f(t)
    return sampler

c, e = tri(c_freq), tri(e_freq)
play(both(note(c,0,1),note(e,1,2)))
g, low_g = tri(g_freq), tri(g_freq / 2)

### NOW THE TIME FOR MARIO!! ###
def mario_at(octave):
    c, e = tri(octave * c_freq), tri(octave * e_freq)
    g, low_g = tri(octave * g_freq), tri(octave * g_freq / 2)
    return mario(c,e,g,low_g)

def mario(c,e,g,low_g):
    z = 0
    song = note(e,z,z+1/8)
    z += 1/8
    song = both(song, note(e,z,z+1/8))
    z += 1/4
    song = both(song, note(e,z,z+1/8))
    z += 1/4
    song = both(song, note(c,z,z+1/8))
    z += 1/8
    song = both(song, note(e,z,z+1/8))
    z += 1/4
    song = both(song, note(g,z,z+1/4))
    z += 1/2
    song = both(song, note(low_g,z,z+1/8))
    z += 1/2
    return song

play(both(mario_at(2), mario_at(1)))

By the example of **sound**, we figure out that higher-order functions is a highly abstracted method of programming. We can apply this clever idea in practice.  
**AT LEAST** We can play a wonderful mario music!

#### 2. Function Abstraction [(Slide)](https://cs61a.org/assets/slides/07-Functional_Abstraction_1pp.pdf) ####

##### 2.1 Lambda Function Environment #####  

!["aha"](../Sources/lec_07.png)

In [4]:
# A good example to show frame structure: 
#   in <lambda y: a + y>, a=3 because it is not in the body of any function and is in global frame.
#   in <f(...)(a)>, a=3 because it is also in the global frame.
#   but in f(g), it returns 2*g(y) because in this local frame of f(g), a = 2

a = 3
def f(g):
    a = 2
    return lambda y: a * g(y)

f(lambda y: a + y)(a) # This always returns <a_in * (a_out + y)>, where y <- a_out


12

##### 2.2 Return and Control  #####

`Return` is seen usually when a call expression is ended, and we would expect some 'output' from this call expression. Once `Return` happens nothing would be executed in this function.

In control, we see `if` statement, it execute "if" part if the cond is satisfied and skip "else" part.  
How about write it in a call expression? See the code below.

In [8]:
def if_(cond, t, f):
    """The function returns t if cond is True, else it returns f."""
    if cond:
        return t
    else:
        return f

from math import sqrt

def real_sqrt(x):
    """Return the real part of square root of x."""
    return if_(x>=0, sqrt(x), 0)

# real_sqrt(-16)  # This would cause ValueError: math domain error,
# because call expression evaluate all arguments even before executing the body of it.

##### 2.3 Logical Expression #####

Some **evaluating rules** in Python:  
1. `con_left and con_right`: First evaluate `con_left`, if `True`, then evaluate `con_right`.
2. `con_left or con_right`: First evaluate `con_left`, if `False`, then evaluate `con_right`.

This is useful in Python to avoid some crashes when we really executing some functions.

##### 2.4 Functional Abstractions #####

We use a function in other functions, what would Python need for this kind of abstraction?  

In [10]:
def square(x):
    return x*x

def sum_square(x, y):
    return square(x) + square(y)

sum_square(2,3)

13

What does `sum_square` need to know about `square` ?  
1. `square` takes one argument  
2. `square` has the functionality to compute the square of a number.
3. `square` does not need to know the **intrinsic name** or **how it is implemented**

##### 2.5 Errors and Tracebacks #####

Error comes in 3 forms:
1. Syntax Error, those are detected before executing the code.
2. Runtime Error, those found out during execution. It has a report to tell the line error happens.
3. Behavior Error, the function just do the wrong thing.

#### 3. Function Examples [(Slides)](https://cs61a.org/assets/slides/08-Function_Examples_1pp.pdf) ####

##### 3.1 Decorator  #####

In [1]:
def trace1(fn):
    """Return a reversion of fn that print fuction and arguments when called.
    
    fn - a function of 1 argument.
    """
    def traced(x):
        print("Calling", fn, 'on argument', x)
        return fn(x)
    return traced

def square(x):
    return x*x

square = trace1(square)

# When you use @ decorator, the following function will be part of a decorated function,
# bounded to the function name again.
@trace1
def sum_squares_up_to(n):
    k = 1 
    total = 0
    while k <= n:
        total, k = total+square(k), k+1
    return total

In [2]:
sum_squares_up_to(5)

Calling <function sum_squares_up_to at 0x0000022AACB49DC8> on argument 5
Calling <function square at 0x0000022AACB4E1F8> on argument 1
Calling <function square at 0x0000022AACB4E1F8> on argument 2
Calling <function square at 0x0000022AACB4E1F8> on argument 3
Calling <function square at 0x0000022AACB4E1F8> on argument 4
Calling <function square at 0x0000022AACB4E1F8> on argument 5


55

#### 4. Recursion [(Slides1)](https://cs61a.org/assets/slides/09-Recursion_1pp.pdf) [(Slides2)](https://cs61a.org/assets/slides/10-Tree_Recursion_1pp.pdf) ####

##### 4.1 Self-Reference #####

A function can refer to **its own name.**  
![-w10](../Sources/lec-09.jpg)

In [2]:
def print_all(x):
    print(x)
    return print_all

print_all(1)(3)(5)(7)

1
3
5
7


<function __main__.print_all(x)>

In [3]:
def print_sums(x):
    print(x)
    def next_sum(y):
        return print_sums(x+y)
    return next_sum

print_sums(1)(3)(5)

1
4
9


<function __main__.print_sums.<locals>.next_sum(y)>

##### **4.2 Recursive Functions** #####

**Recursive Function:** A function that the body of it calls itself (either directly or indirectly). So executing recursive functions requires calling it again.  
**Enviornment Diagram** of Recursive Function is important to understand.

In [8]:
def split(n):
    """Split positive n into all but its last digit and its last digit."""
    return n // 10, n % 10

# The def statement header is the same.
# Check basic case without recursive calls.
# Do recursive part in recursive cases.

def sum_digits(n):
    """Return the sum of digits of positive integer n."""
    if n < 10:
        return n
    else:
        all_but_last, last = split(n)        
        return sum_digits(all_but_last) +  last 

sum_digits(1229203)

19

In [13]:
# This is an example of mutual recursion.

def luhn_sum(n):
    if n < 10:
        return n
    else:
        all_but_last, last = split(n)
        return luhn_sum_double(all_but_last) + last

def luhn_sum_double(n):
    all_but_last, last = split(n)
    luhn_digit = sum_digits(2 * last)
    if n < 10:
        return luhn_digit
    else:
        return luhn_sum(all_but_last) + luhn_digit

luhn_sum(6216630100000813263) #This is my unionpay debit card number.

50

##### **4.3 Tree Recursion** #####

**Order of recursive functions** is important in understanding the behavior of recursion.  
An important rule: When making a function call you must do anything else **after** it returns something.  
eg: If (a) calls other functions, then it should wait until this call returns something and then do next things.

In [15]:
def cascade(n):
    if n < 10:
        print(n)
    else:
        print(n)
        cascade(n//10)
        print(n)

cascade(12345)

12345
1234
123
12
1
12
123
1234
12345
